<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plan d'expérience:

Régression

      - A.baseline radar cmls -> pluvios

Fusion, fusion multitâche

    - B.baseline radar + pluvio 1 min + cmls -> pluvios + cmls

    - C.tâches aux. 1.  A + tâche aux : -> val cml x masque segment x masque radar + val pluvio x masque pluvio x masque radar

    - D. tâches aux 2. tâche reconstruction PPI.
      - idée : f(radar[random_sample1], w) = y[70 x 64 x 64].  
         ft de coût : ||f(radar[random_sample1])[random_sample1_bar] - target ||
         + loss adversariale -> impossibilité de retrouver les indices renseignés

    - E. tâche aux 1 + tâche aux 2. A + (B) + c

Régression faiblement supervisée

    - E. introduction d'un bruit dans les CMLs et les pluvios

    - F. correction de l'effet du bruit dans les CMLs dans la loss (weak. sup.)

    - G. correction de l'effet du bruit par un réseau auxilaire (// denoising)





# Partie I : régression

In [1]:
! git clone https://github.com/nanopiero/fusion.git

Cloning into 'fusion'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 101 (delta 59), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (101/101), 1002.68 KiB | 11.02 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

from random import randint
import os

# imports des fichiers locaux
os.chdir('fusion')
import utile_fusion
# import importlib
# importlib.reload(utile_fusion)

# Import des fonctions génératrices exploitées à l'échelle de l'image
from utile_fusion import spatialized_gt, create_cmls_filter
# Import des fonctions utilisées à l'échelle du batch, sur carte GPU
from utile_fusion import point_gt, segment_gt, make_noisy_images
#Import des fonctions de visualisation
from utile_fusion import set_tensor_values2, plot_images
from utile_fusion import FusionDataset


In [3]:
# config:
npoints = 5
npairs = 10
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

In [4]:
# Dataset, DataLoader

dataset = FusionDataset(length_dataset=length_dataset, npairs=npairs, nsteps=60,
                        ndiscs=ndiscs, size_image=size_image)

from torch.utils.data import DataLoader
loader = DataLoader(dataset, batch_size=64, num_workers=4)

In [5]:
# Essai avec un FCN (encodage simple)



################################   UNet (parties)###############################
import torch
import torch.nn as nn
import torch.nn.functional as F

class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x



class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(Up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear')
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, kernel_size=2, stride=2)

        self.conv = double_conv(2*in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)


    def forward(self, x):
        x = self.conv(x)
        return x


################################################################################
########################################   Mini Unet  ##########################

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes,size=64):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, size)
        self.down1 = Down(size, 2*size)
        self.down2 = Down(2*size, 4*size)
        self.down3 = Down(4*size, 8*size)
        self.down4 = Down(8*size, 8*size)
        self.up1 = Up(8*size, 4*size)
        self.up2 = Up(4*size, 2*size)
        self.up3 = Up(2*size, size)
        self.up4 = Up(size, size)
        self.outc = outconv(size, n_classes)
        self.outc2 = outconv(size, n_classes)
        self.n_classes=n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return   x


# Exemple d'instanciation :
ch_in = 14
ch_out = 1
size = 16

model = UNet(ch_in, ch_out, size)




In [16]:
print(images.shape)
bs, nsteps, S, _ = images.shape
_, nlinks, _, _ = filters.shape

filters[filters == 0] = torch.nan
filtered_images = images.unsqueeze(dim=2) * \
                  filters.unsqueeze(dim=1)
# on ajoute 1. pour distinguer du cas == 0
sampled_values = torch.nanmean(filtered_images,\
                  dim=(3,4)) + 1.

print(sampled_values.shape)
filters[filters != filters] = 0
filters = filters.unsqueeze(1)
print(filters.shape, sampled_values.view(bs, nsteps, nlinks, 1, 1).shape)
filters = filters * sampled_values.view(bs, nsteps, nlinks, 1, 1)
filters = filters.sum(dim=2)
print(filters.shape)

torch.Size([64, 60, 64, 64])
torch.Size([64, 60, 10])
torch.Size([64, 1, 10, 64, 64]) torch.Size([64, 60, 10, 1, 1])
torch.Size([64, 60, 64, 64])


In [39]:

bs, nsteps, S, _ = images.shape
flat_images = images.view(bs, nsteps, S * S)
# Randomly sample M indices for each image in the batch
indices = torch.randint(0, S * S, (bs, npoints), \
                        device=images.device)

# Gather the values from these indices for all images
sampled_values = torch.gather(flat_images, 2, indices.unsqueeze(dim=1).repeat([1,nsteps,1]))
print(sampled_values.shape)


point_measurements = torch.zeros(images.numel()).to(device)
# point_measurements[indices]

torch.Size([64, 60, 5])


In [37]:
images.numel()

15728640

In [30]:
pos = torch.randint(0, S, (3, npoints, npoints), \
                        device=images.device)
pos

tensor([[[3, 3, 1, 2, 0],
         [2, 1, 1, 2, 0],
         [3, 3, 3, 2, 1],
         [3, 2, 0, 0, 0],
         [0, 2, 2, 3, 2]],

        [[0, 0, 0, 2, 3],
         [1, 2, 0, 0, 3],
         [0, 1, 0, 3, 2],
         [0, 2, 1, 3, 3],
         [2, 0, 2, 1, 1]],

        [[3, 2, 3, 3, 1],
         [3, 0, 1, 3, 1],
         [2, 2, 3, 2, 0],
         [3, 2, 0, 3, 3],
         [2, 3, 1, 2, 0]]])

In [33]:
def segment_gt_fcn(images, pairs, filters):
  bs, nsteps, S, _ = images.shape
  _, nlinks, _, _ = filters.shape

  filters[filters == 0] = torch.nan
  filtered_images = images.unsqueeze(dim=2) * \
                    filters.unsqueeze(dim=1)
  # on ajoute 1. pour distinguer du cas == 0
  sampled_values = torch.nanmean(filtered_images,\
                    dim=(3,4)) + 1.

  filters[filters != filters] = 0
  filters = filters.unsqueeze(1)
  filters = filters * sampled_values.view(bs, nsteps, nlinks, 1, 1)
  filters = filters.sum(dim=2)
  return filters


def point_gt_fcn(images, npoints=10):
  bs, nsteps, S, _ = images.shape
  flat_images = images.view(bs, nsteps, S * S)
  # Randomly sample M indices for each image in the batch
  indices = torch.randint(0, S * S, (bs, npoints), \
                          device=images.device)

  # Gather the values from these indices for all images
  sampled_values = torch.gather(flat_images, 2, indices.unsqueeze(dim=1).repeat([1,nsteps,1]))

  # Calculate coordinates from indices
  rows = indices // S
  cols = indices % S
  print(rows)
  point_measurements = torch.zeros(images.shape).to(device)
  for j in range(rows.shape[0]):
    for i in range(rows.shape[1]):
     point_measurements[j,:,rows[j,i],cols[j,i]] = images[:,:,rows[i],cols[i]]
  return point_measurements


# Baseline with a FCN

for i, (images, pairs, filters) in enumerate(loader):

  # ground truth (not usable)
  images = images.clone().detach().float().to(device)

  # pseudo CMLs
  pairs = pairs.clone().detach().float().to(device)
  filters = filters.clone().float().detach().to(device)

  # for transformers :
  # segment_measurements = segment_gt(images, pairs, filters)

  segment_measurements = segment_gt_fcn(images, pairs, filters)

  # pseudo pluvios
  point_measurements = point_gt_fcn(images, npoints=5)

  # pseudo radar
  noisy_images = make_noisy_images(images)

  # prepare inputs and targets
  inputs = torch.cat([noisy_images, segment_measurements], dim=1)
  targets = point_measurements


  optimizer.zero_grad()  # Zero the gradients
  outputs = model(inputs)  # Forward pass
  loss = criterion(outputs, targets)  # Compute the loss
  loss.backward()  # Backward pass
  optimizer.step()  # Update the weights
  aaa

tensor([[56, 45,  3,  4, 61],
        [37, 25, 26, 57, 12],
        [38,  9, 29, 60, 36],
        [49, 18, 22, 13, 58],
        [49, 51, 61, 56,  0],
        [53, 63, 55, 36, 32],
        [61,  5, 26, 31, 34],
        [43, 12, 34, 19, 40],
        [14,  2, 42, 35, 16],
        [34, 12,  6, 38, 31],
        [49,  1,  7, 57, 63],
        [34, 16, 58, 44, 49],
        [60, 31, 13, 59, 14],
        [60, 57, 22, 46, 19],
        [29, 61,  3,  1, 41],
        [13, 26, 54, 10, 57],
        [ 2, 20, 34, 40, 14],
        [50, 44, 47, 52,  3],
        [42, 20,  6,  3, 35],
        [ 2, 63, 28, 58, 10],
        [ 4,  7, 20, 27,  3],
        [26, 14, 10, 37, 41],
        [21, 50, 18, 39, 18],
        [29, 40, 35, 41, 22],
        [48, 45, 28, 46, 15],
        [61, 40, 51, 62,  7],
        [ 5, 25, 25,  0, 48],
        [35, 11, 12, 20, 13],
        [19, 50,  3, 45, 36],
        [62, 28, 17, 33, 30],
        [56, 48, 39, 12, 32],
        [57, 53, 13,  4, 56],
        [58, 50,  8, 61, 24],
        [5

RuntimeError: expand(torch.cuda.FloatTensor{[64, 60, 5]}, size=[60]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (3)

In [ ]:
# Exemple / tracés

images = set_tensor_values2(images, point_measurements)
plot_images(images[0,...].cpu().numpy() + filters[0,...].cpu().numpy().sum(axis=0),
            noisy_images[0,...].cpu().numpy(),
            point_measurements[0,...].cpu().numpy(),
            segment_measurements[0,...].cpu().numpy())

In [22]:
# Archi

# cas d'un "fusion transformer 4d"
# Paramètres du modèle :
image_size = [64,64]
channels = 1
patch_size = 4
d_model = 120
mlp_expansion_ratio = 4
d_ff = mlp_expansion_ratio * d_model
n_heads = 4
n_layers = 12


model = FusionTransformer2dplus(image_size, patch_size, n_layers, d_model, d_ff, n_heads, channels=1)


In [21]:
! pip install einops

#Transformer 2d, time = channels
from utile_Transformers import Block, Decoder
import torch.nn as nn

class UnifiedEmbedding2dplus(nn.Module):
  # le temps ici est compté comme un channel
    def __init__(self, d_model, patch_size, channels, nsteps):
        super().__init__()
        self.d_model = d_model
        self.patch_size = patch_size
        self.channels = channels
        self.nsteps = nsteps
        self.dim_modality = 4
        # Positional embedding for coordinates
        self.coord_embed = nn.Linear(2, d_model // 3)

        # Modality specific embeddings
        self.patch_modality = nn.Parameter(torch.randn(self.dim_modality))
        self.point_modality = nn.Parameter(torch.randn(self.dim_modality))
        self.segment_modality = nn.Parameter(torch.randn(self.dim_modality))

        # Feature embedding for radar image patches
        self.patch_feature_embed = nn.Conv2d(channels, d_model - self.dim_modality \
                - 2 * (d_model // 3), kernel_size=patch_size, stride=patch_size)

        # Feature embedding for point and segment rain rates
        self.punctual_rain_rate_embed = nn.Linear(nsteps, d_model - self.dim_modality \
                                                  - 2 * (d_model // 3))
        self.integrated_rain_rate_embed = nn.Linear(nsteps, d_model - self.dim_modality \
                                                   - 2 * (d_model // 3))

    def forward(self, image, points, segments):
        B, C, H, W = image.shape
        device = image.device
        # print("Image shape:", image.shape)

        # Embedding patches
        patch_embeddings = self.patch_feature_embed(image).flatten(2).transpose(1, 2)
        # print("Patch embeddings shape:", patch_embeddings.shape)

        # Create grid for patches
        grid_x, grid_y = torch.meshgrid(torch.arange(0, H, self.patch_size), torch.arange(0, W, self.patch_size), indexing='ij')
        grid_x = grid_x.to(device)
        grid_y = grid_y.to(device)
        upleft = torch.stack((grid_x.flatten(), grid_y.flatten()), dim=-1).float()
        downright = torch.stack((grid_x.flatten() + self.patch_size, grid_y.flatten() + self.patch_size), dim=-1).float()
        # erreur chatGPT !! patch_pos_embeddings = self.coord_embed(upleft) + self.coord_embed(downright)
        patch_pos_embeddings = torch.cat([self.coord_embed(upleft), self.coord_embed(downright)], dim=-1)
        patch_pos_embeddings = patch_pos_embeddings.repeat(B, 1, 1)
        # print("Patch positional embeddings shape:", patch_pos_embeddings.shape)

        patch_embeddings = torch.cat([patch_embeddings, patch_pos_embeddings, self.patch_modality.unsqueeze(0).expand(B, patch_embeddings.size(1), -1)], dim=-1)
        # print("Final patch embeddings shape:", patch_embeddings.shape)

        # Embedding points
        point_pos_embeddings = self.coord_embed(points[..., :2].float())
        # print("Point positional embeddings shape:", point_pos_embeddings.shape)

        point_feature_embeddings = self.punctual_rain_rate_embed(points[..., 2:].float())
        point_embeddings = torch.cat([point_feature_embeddings, point_pos_embeddings, point_pos_embeddings, self.point_modality.unsqueeze(0).expand(B, points.size(1), -1)], dim=-1)
        # print("Final point embeddings shape:", point_embeddings.shape)

        # Embedding segments
        seg_pos_embeddings0 = self.coord_embed(segments[..., :2].float())
        seg_pos_embeddings1 = self.coord_embed(segments[..., 2:4].float())
        segment_feature_embeddings = self.integrated_rain_rate_embed(segments[..., 4:].float())
        segment_embeddings = torch.cat([segment_feature_embeddings, seg_pos_embeddings0, seg_pos_embeddings1, self.segment_modality.unsqueeze(0).expand(B, segments.size(1), -1)], dim=-1)
        # print("Final segment embeddings shape:", segment_embeddings.shape)

        # Concatenate all embeddings
        embeddings = torch.cat([patch_embeddings, point_embeddings, segment_embeddings], dim=1)

        return embeddings

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
        nn.init.constant_(m.bias, 0)
        nn.init.constant_(m.weight, 1.0)

def trunc_normal_(tensor, mean=0, std=1):
    nn.init.trunc_normal_(tensor, mean=mean, std=std)


class FusionTransformer2dplus(nn.Module):
    def __init__(
        self,
        image_size,
        patch_size,
        n_layers,
        d_model,
        d_ff,
        n_heads,
        channels=1,
        nsteps=60
    ):
        super().__init__()
        self.ue = UnifiedEmbedding2dplus(d_model, patch_size, channels, nsteps)
        self.patch_size = patch_size
        self.n_layers = n_layers
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads


        self.blocks = nn.ModuleList(
            [Block(d_model, n_heads, d_ff) for _ in range(n_layers)]
        )
        self.norm = nn.LayerNorm(d_model)

        self.apply(init_weights)

        self.decoder = Decoder(patch_size, d_model)

    def forward(self, x, y, z):
        # Embed signal
        x = self.ue(x, y, z)  # (B, N, D)

        # Process through each transformer block
        for block in self.blocks:
            x = block(x)

        # Apply final normalization
        x = self.norm(x)
        x = x[:,:256,:]

        x = self.decoder(x)

        return x


In [26]:
model = model.to(device)

model(noisy_images, point_measurements, segment_measurements)

RuntimeError: Given groups=1, weight of size [36, 1, 4, 4], expected input[1, 12, 64, 64] to have 1 channels, but got 12 channels instead

In [ ]:
# Exemple / tracés
images = set_tensor_values2(images, point_measurements)
plot_images(images[0,...].cpu().numpy() + filters[0,...].cpu().numpy().sum(axis=0),
            noisy_images[0,...].cpu().numpy(),
            point_measurements[0,...].cpu().numpy(),
            segment_measurements[0,...].cpu().numpy())

tensor(0.1194, device='cuda:0')